In [592]:
import numpy as np
import nltk
import itertools
#nltk.download('all')
from nltk.tokenize import sent_tokenize

In [593]:
# softmax: take each COLUMN as a score vector
# X: c x n np.ndarray
# return a c x n ndarray
# Caveat: if X is 1 * n array, it will apply on "row" still
def softmax_by_col(X):
    M = X.copy()
    M -= M.max(axis=0)  # for each column, substract the max over row
    eM = np.exp(M) 
    return eM / eM.sum(axis=0)  # softmax for each column

In [594]:
# Caveat: if X is 1 * n array, it will return the same 1 * n array
def softmax(X, by='col'):
    if by == 'col':
        return softmax_by_col(X)
    elif by == 'row':
        return softmax_by_col(X.T).T
    else:
        raise        

In [596]:
# y: nSamp x dim, each row is the real prob.
# y_pred: nSamp x dim, each row is the predicted prob.
# return: sum loss for all samples
def sum_log_loss(y, y_pred):
    return - ( y * np.log(y_pred + 1e-9) ).sum(axis=1).sum()

In [602]:
y = np.asarray([[0., 1., 0., 0.], [0.3, 0.4, 0.2, 0.1]])
y_pred = np.asarray([[0.1, 0.9, 0., 0.], [0.3, 0.4, 0.2, 0.1]])
print sum_log_loss(y, y_pred)
print -np.log(0.9)

1.38521473638
0.105360515658


In [663]:
# x is a sequence/list of index (index in 0 - dim-1), 1 * n
# return: e, n * dim
def one_hot_encode(x, dim):
    #print x, dim
    n = len(x)
    #print n
    e = np.zeros( (n, dim) )
    e[xrange(n), x] += 1
    #print e.shape
    return e

In [604]:
# e: an one-hot code, e.g. [0, 0, 1, 0]
# return: index, e.g. 2
def one_hot_decode(e):
    return np.argmax(e)

# Load and Prepare Training Data

In [605]:
voc_sz = 2000
unknown = "#"
sent_start = "^"
sent_end = "@"

### Load text files

In [606]:
# Read the data and append SENTENCE_START and SENTENCE_END tokens
print "Reading training file..."
with open('sayings.txt', 'rb') as f:
    # body = f.read()
    sents = f.readlines()

Reading training file...


In [607]:
sents = [s.lower().rstrip() for s in sents if len(s)>2]

In [608]:
sents = ["%s %s %s" % (sent_start, s, sent_end) for s in sents]

In [609]:
print "Parsed %d sentences." % (len(sentences))
sents[3]

Parsed 1129 sentences.


'^ a cat may look at a king @'

In [610]:
# Tokenize the sentences into words
sents_t = [nltk.word_tokenize(s) for s in sents]  # a list of sent_t, i.e. list of words
print len(sents_t), " tokenized sentences."

567  tokenized sentences.


In [611]:
print "\nExample sentence: %s" % sents[0]
print "\nExample sentence after Pre-processing: %s" % sents_t[546]


Example sentence: ^ a bad penny always turns up @

Example sentence after Pre-processing: ['^', 'work', 'expands', 'so', 'as', 'to', 'fill', 'the', 'time', 'available', '@']


In [614]:
sents_t[546]

['^',
 'work',
 'expands',
 'so',
 'as',
 'to',
 'fill',
 'the',
 'time',
 'available',
 '@']

### Build Vocabulary

In [615]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*sents_t)) # a FreqDist object

In [616]:
print "Example word_freq itesm: ", word_freq.items()[:3]
print "Found %d unique words tokens." % len(word_freq.items())

Example word_freq itesm:  [('stones', 3), ('all', 32), ('forget', 1)]
Found 1196 unique words tokens.


In [617]:
voc_sz = min( [voc_sz, len(word_freq.items())] )
print "voc_sz = ", voc_sz

voc_sz =  1196


In [620]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common( voc_sz ) # a list of tuple (word, cnt)
print vocab[:10]

[('^', 567), ('@', 567), ('the', 239), ('a', 183), ('is', 142), ('to', 84), (',', 84), ('you', 72), ("'s", 69), ('and', 68)]


### Training Data

In [621]:
# word -> an index in vocab, index in [0, voc_sz)
word_to_idx = dict([(w, idx) for idx, (w, cnt) in enumerate(vocab)])

In [622]:
word_to_idx['^']

0

In [623]:
# Create the training data
# X: each row is a training sample, i.e., an input sequence, or, i.e., a sentence. Each word is an index (0 - voc_sz-1)
# y: each row is a output, i.e., a shift-1-word sequence
X_train = np.asarray([ [word_to_idx[w] for w in sent_t[:-1]] for sent_t in sents_t ])  # n sents_t
Y_train = np.asarray([ [word_to_idx[w] for w in sent_t[1:]] for sent_t in sents_t ])   # n sents_t

In [624]:
X_train[:5]

array([[0, 3, 92, 136, 47, 511, 135], [0, 3, 436, 91, 19, 617],
       [0, 3, 182, 14, 2, 160, 4, 68, 69, 14, 2, 853],
       [0, 3, 145, 71, 231, 57, 3, 383],
       [0, 3, 484, 4, 98, 16, 553, 16, 72, 1020, 1133]], dtype=object)

In [625]:
Y_train[:5]

array([[3, 92, 136, 47, 511, 135, 1], [3, 436, 91, 19, 617, 1],
       [3, 182, 14, 2, 160, 4, 68, 69, 14, 2, 853, 1],
       [3, 145, 71, 231, 57, 3, 383, 1],
       [3, 484, 4, 98, 16, 553, 16, 72, 1020, 1133, 1]], dtype=object)

# Training RNN

In [626]:
class RNN:
     def __init__(self, dim, dim_h=100, bptt_truncate=4):
        # Assign instance variables
        self.dim = dim # dim. of input/output vec, e.g. the embedding vec. of a word in a sequence
        self.dim_h = dim_h # hidden layer
        self.bptt_truncate = bptt_truncate
        # U: input vec to hidden states, dim * dim_h, (so we can dot(x, U))
        self.U = np.random.randn( dim, dim_h ) * np.sqrt( 2. / ( dim * dim_h ) )
        # V: hidden states to output vec, dim_h * dim (so we can dot(V, h) )
        self.V = np.random.randn( dim_h, dim ) * np.sqrt( 2. / ( dim_h * dim ) )
        # W: (prev) hidden state to (next) hidden state 
        self.W = np.random.randn( dim_h, dim_h ) * np.sqrt( 2. / ( dim_h * dim_h ) )

In [627]:
# x: T * dim, a input sequence of vectors, [x1, x2, ..., x_t], each row is a 1 * dim vector
# return: o, output softmax scores, T * dim
#         s, hidden states, (T+1) * dim_h. s[-1] is initial state, all zeros
def forwardProp(self, x):
    T = len(x)
    # Save all hidden states for bptt.
    # Add one initial hidden state, which are 0
    s = np.zeros( (T + 1, self.dim_h) ) # s[-1] are zeros (1 * dim_h)
    o = np.zeros( (T, self.dim) )
    for t in xrange(T):
#         print "t = ", t
#         print "x[t]", x[t]
#         print "x[t].shap", x[t].shape
#         print "U", self.U.shape
#         print "s", s[t-1].shape
#         print "W", self.W.shape
        s[t] = np.tanh( np.dot( x[t], self.U ) + np.dot( s[t-1], self.W ) ) # 1 * dim_h
        o[t] = softmax( np.dot( s[t], self.V ), by='row' )  # 1 * dim 
    return [o, s]
RNN.forwardProp = forwardProp

In [628]:
# x: T * dim, a input sequence of vectors, [x1, x2, ..., x_t], each row is a 1 * dim vector
# return: labels, 1 * T, 
#         For each step t, label is the index of the dimension with max value in the output softmax score,
def predict(self, x):
    o, s = self.forwardProp(x)
    return np.argmax(o, axis=1)
RNN.predict = predict

In [629]:
# X: a batch of n sequences, each sequence is a T * dim array. T may vary by sequence
# Y: a batch of n sentences, each sequence is a T * dim array. T may vary by sequence
# return: the sum log loss on all sequences
def total_loss(self, X, Y):
    n = len(X) # num. of training sequences
    sum_loss = 0.
    for i in xrange(n): # for each sequence
        x = X[i]  # a sequence, T * dim
        y = Y[i]  # a sequence, T * dim
        o, s = self.forwardProp( x ) # T * dim
        loss = sum_log_loss( y, o ) # sum loss in this sequence
        sum_loss += loss
    return sum_loss  # sum loss on all sequences of vectors
RNN.total_loss = total_loss

In [630]:
# X: a batch of n sequences, each sequence is a T * dim array. T may vary by sequence
# Y: a batch of n sentences, each sequence is a T * dim array. T may vary by sequence
# return: the average log loss for each vector
def loss(self, X, Y):
    n = len(X) # num. of training sequences
    sum_loss = 0.
    for i in xrange(n): # for each sequence
        x = X[i]  # a sequence, T * dim
        #print x.shape
        y = Y[i]  # a sequence, T * dim
        #print y.shape
        o, s = self.forwardProp( x ) # T * dim
        #print o.shape
        loss = sum_log_loss( y, o ) # sum loss in this sequence
        #print loss
        sum_loss += loss
    n_vec = np.sum( [len(y) for y in Y] )  
    #print n_vec
    return sum_loss / n_vec  # avg. loss per vector
RNN.loss = loss

In [631]:
### !!!  seeems only for one-hot encoding 
# x: a sequence of vec, T * dim
# y: a sequence of vec, T * dim
def bptt(self, x, y):
    T = len(y)
    o, s = self.forwardProp(x)  # o: T * dim,  s: T * dim_h
    dU = np.zeros_like(self.U)  # dU is actually dL/dU
    dV = np.zeros_like(self.V)
    dW = np.zeros_like(self.W)
    idx_x = np.argmax(x, axis=1) # 1 * T, each entry is an index
    idx_y = np.argmax(y, axis=1) # 1 * T, each entry is an index
    delta_o = o  # T * dim
    delta_o[np.arange(len(idx_y)), idx_y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        #print dV.shape  # dim_h * dim
        dV += np.outer(s[t].T, delta_o[t])
        # Initial delta calculation
        delta_t = np.dot(delta_o[t], self.V.T) * (1 - (s[t] ** 2))   
        # Backpropagation through time (for at most self.bptt_truncate steps)
        steps = np.arange(max(0, t-self.bptt_truncate), t+1)
        for bptt_step in steps[::-1]:
            dW += np.outer(s[bptt_step-1].T, delta_t)              
            dU[idx_x[bptt_step], :] += delta_t
            # Update delta for next step
            delta_t = np.dot(delta_t, self.W.T) * (1 - s[bptt_step-1] ** 2)
    return [dU, dV, dW]
RNN.bptt = bptt

In [632]:
import operator
# x: a sequence of vec, T * dim
# y: a sequence of vec, T * dim
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = self.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            #print [x].shape
            #print [y].shape
            gradplus = self.total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = self.total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error >= error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
RNN.gradient_check = gradient_check

In [633]:
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 10
np.random.seed(10)
model = RNN(grad_check_vocab_size, 5, bptt_truncate=1000)

#print "U", model.U.shape
model.gradient_check( one_hot_encode( [0,1,2,3], grad_check_vocab_size), 
                      one_hot_encode( [1,2,3,4], grad_check_vocab_size)
                    )

Performing gradient check for parameter U with size 50.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 50.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 25.
Gradient check for parameter W passed.


/Users/sxue/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:33: RuntimeWarning: invalid value encountered in double_scalars


In [634]:
# Performs one step of SGD on 1 training sequence
# x: a sequence of vec, T * dim
# y: a sequence of vec, T * dim
def sgd_update(self, x, y, step):
    # Calculate the gradients
    dU, dV, dW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U += - step * dU
    self.V += - step * dV
    self.W += - step * dW
RNN.sgd_update = sgd_update

In [635]:
from datetime import datetime
# Training Loop
# - X_train: The training data set, batch of sequences
# - y_train: The training data labels, batch
# - step: learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
def train(self, X_train_encode, Y_train_encode, step=0.005, nepoch=100):
    losses = []     # keep track of the losses
    n_sample_seen = 0  # samples (sequences) have seen
    for epoch in range(nepoch):
        if (epoch % 5 == 0):
            loss = self.loss(X_train_encode, Y_train_encode)
            losses.append( (n_sample_seen, loss) )
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, n_sample_seen, epoch, loss)
            # Adjust the learning rate if loss increases
            if ( len(losses) >= 2 and losses[-1][1] >= losses[-2][1]):
                step *= 0.5 
                print "Setting learning rate to %f" % step

        n = len(Y_train_encode)
        for i in xrange(n): # train sample by sample, i.e. batch=1
            self.sgd_update(X_train_encode[i], Y_train_encode[i], step)
            n_sample_seen += 1
RNN.train = train

In [636]:
np.random.seed(10)
print voc_sz
rnn = RNN(dim=voc_sz)

1196


In [637]:
print X_train.shape
print Y_train.shape

(567,)
(567,)


In [638]:
X_train_encode = []
for x in X_train: # a sequence of indices
    x_encode = one_hot_encode( x, voc_sz ) # T * dim
    X_train_encode.append( x_encode )
Y_train_encode = []
for y in Y_train: # a sequence of indices
    y_encode = one_hot_encode( y, voc_sz ) # T * dim
    Y_train_encode.append( y_encode )

In [639]:
%timeit rnn.sgd_update(X_train_encode[10], Y_train_encode[10], step=0.005)

100 loops, best of 3: 7.15 ms per loop


In [701]:
rnn.train(X_train_encode, Y_train_encode, step=0.005, nepoch=200)

2016-11-12 15:58:20: Loss after num_examples_seen=0 epoch=0: 2.966479
2016-11-12 15:58:43: Loss after num_examples_seen=2835 epoch=5: 3.335902
Setting learning rate to 0.002500
2016-11-12 15:59:06: Loss after num_examples_seen=5670 epoch=10: 2.890300
2016-11-12 15:59:27: Loss after num_examples_seen=8505 epoch=15: 2.768825
2016-11-12 15:59:49: Loss after num_examples_seen=11340 epoch=20: 2.743475
2016-11-12 16:00:11: Loss after num_examples_seen=14175 epoch=25: 2.741667
2016-11-12 16:00:36: Loss after num_examples_seen=17010 epoch=30: 2.739850
2016-11-12 16:00:59: Loss after num_examples_seen=19845 epoch=35: 2.707654
2016-11-12 16:01:21: Loss after num_examples_seen=22680 epoch=40: 2.695286
2016-11-12 16:01:43: Loss after num_examples_seen=25515 epoch=45: 2.678172
2016-11-12 16:02:09: Loss after num_examples_seen=28350 epoch=50: 2.640412
2016-11-12 16:02:31: Loss after num_examples_seen=31185 epoch=55: 2.603842
2016-11-12 16:02:54: Loss after num_examples_seen=34020 epoch=60: 2.599846


In [702]:
rnn.train(X_train_encode, Y_train_encode, step=0.005, nepoch=1000)

2016-11-12 16:13:06: Loss after num_examples_seen=0 epoch=0: 2.230186
2016-11-12 16:13:26: Loss after num_examples_seen=2835 epoch=5: 3.806476
Setting learning rate to 0.002500
2016-11-12 16:13:46: Loss after num_examples_seen=5670 epoch=10: 2.865893
2016-11-12 16:14:06: Loss after num_examples_seen=8505 epoch=15: 2.388203
2016-11-12 16:14:27: Loss after num_examples_seen=11340 epoch=20: 2.309785
2016-11-12 16:14:47: Loss after num_examples_seen=14175 epoch=25: 2.165267
2016-11-12 16:15:07: Loss after num_examples_seen=17010 epoch=30: 2.114324
2016-11-12 16:15:26: Loss after num_examples_seen=19845 epoch=35: 2.071835
2016-11-12 16:15:47: Loss after num_examples_seen=22680 epoch=40: 2.051029
2016-11-12 16:16:06: Loss after num_examples_seen=25515 epoch=45: 2.023273
2016-11-12 16:16:26: Loss after num_examples_seen=28350 epoch=50: 2.017663
2016-11-12 16:16:46: Loss after num_examples_seen=31185 epoch=55: 2.006291
2016-11-12 16:17:05: Loss after num_examples_seen=34020 epoch=60: 2.106739


In [664]:
# x: a list of vectors (one hot encoded words)
# return: a string
def decode_to_sentence(x):
    sent_t = [ vocab[one_hot_decode(e)][0] for e in x ] # a list of words (string)
    return " ".join(sent_t)

In [665]:
print sents[3]
print X_train[3]

^ a cat may look at a king @
[0, 3, 145, 71, 231, 57, 3, 383]


In [666]:
one_hot_encode(X_train[3], voc_sz).shape

(8, 1196)

In [667]:
decode_to_sentence( one_hot_encode(X_train[3], voc_sz) )

'^ a cat may look at a king'

In [668]:
decode_to_sentence( one_hot_encode(Y_train[3], voc_sz) )

'a cat may look at a king @'

In [699]:
def generate(self):
    x = one_hot_encode( [word_to_idx[sent_start]], self.dim ) # an encoded sentence
    cnt = 0
    while not one_hot_decode( x[-1] ) == word_to_idx[sent_end]:
        o, s = self.forwardProp( x )  # o: T * dim
        e = np.random.multinomial(1, o[-1]) # a sample from MN distri, e.g. [0, 0, 1, 0] for 4-choice multi-nomial.
        x = np.append( x, [e], axis=0 )
        cnt += 1
        if cnt >= 30:  # avoid too long the sentence is
            break 
    return decode_to_sentence( x )
RNN.generate = generate

In [703]:
num_sents = 20
for i in range(num_sents):
    print rnn.generate()

^ do n't shoot the small @
^ march abhors and april showers bring forth can flowers @
^ dead men tell no tales @
^ one half of the world does not know how the other half cherries @
^ if it laughs n't broke , do n't @
^ master begins at nine @
^ to err is human ; to forgive divine @
^ a picture paints a man words does @
^ power corrupts ; absolute power corrupts absolutely @
^ let not the sun go down on your wrath @
^ see no evil , hear no evil , speak no evil @
^ charity begins at multitude @
^ the proof of the pudding is in you than cream @
^ it 's no use crying over spilt milk @
^ to err is human ; to forgive divine @
^ the opera schemes not the cradle @
^ he who can does , he who can not , teaches @
^ you can choose your much but you ca n't choose your family @
^ better to have loved and lost than never to have loved at all @
^ distance heart enchantment to the view @


In [705]:
import pickle
with open('rnn.pkl', 'wb') as f:
    pickle.dump(rnn, f, pickle.HIGHEST_PROTOCOL)